In [37]:
import os
import itertools
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
path = 'C:/Users/Pragati Shah/subset/'
path1='C:/Users/Pragati Shah/Downloads/sample_10000/sample_10000/'

In [58]:
ref_all=[]
source_all=[]
pmid_all=[]
doi_all=[]
for fn in os.listdir(path1):
    tree = ET.parse(path1+fn)  
    root = tree.getroot()    
    ref_list=[]
    source_list=[]
    pmid=0
    doi=0
    for elem in tree.iter(tag='article-id'):
        a=str(elem.attrib)
        if (a.find('pmid')>-1):        
            pmid=elem.text
        if (a.find('doi')>-1):        
            doi=elem.text
    for a in root.iter('ref'):
        for elem in a.iter('source'):
            source_list.append(elem.text)
        for elem in a.iter('pub-id'):
            ref_list.append(elem.text)
    
    doi_list=np.repeat(doi,len(source_list))
    pmid_list=np.repeat(pmid,len(source_list))
    
    pmid_all.extend(pmid_list)
    doi_all.extend(doi_list)
    ref_all.extend(ref_list)
    source_all.extend(source_list)
    
    
ref_source = pd.DataFrame(list(zip(doi_all,pmid_all,ref_all,source_all)),
              columns=['Doi','Pmid','Ref_Pmid','Journal']) 
    

In [60]:
ref_source.head()

,Doi,Pmid,Ref_Pmid,Journal
0,10.1371/journal.pone.0000100,17183633,2118947,Journal of Neuroscience
1,10.1371/journal.pone.0000100,17183633,4959494,Science
2,10.1371/journal.pone.0000100,17183633,4977839,Journal of Neurophysiology
3,10.1371/journal.pone.0000100,17183633,10884334,Nature Neuroscience
4,10.1371/journal.pone.0000100,17183633,11588197,Journal of Neuroscience


In [64]:
nodes=list(set(pmid_all+ref_all))

In [65]:
subset = ref_source[['Pmid', 'Ref_Pmid']]
tuples = [tuple(x) for x in subset.values]

In [70]:
tuples[2]

('17183633', '4977839')

In [71]:
import networkx as nx
G = nx.Graph()
G=nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(tuples)

In [80]:
deg_sep=nx.shortest_path_length(G, '17183633','11588197')